J'AI MODIFIE LE FICHIER EXCEL AUPARAVANT, EN REMPLACANT TOUTES LES VIRGULES PAR DES POINTS.

In [77]:
# librairies

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from random import shuffle
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, accuracy_score, f1_score, precision_score, recall_score, mean_squared_error

In [78]:
# import des données

pd.set_option('display.max_columns', None)
yt = pd.read_csv('challenge_youtube_toxic.csv', encoding = 'latin1', engine = 'python', sep=';')

In [79]:
# Préparation des données (input et output)
# On utilise LabelEncoder pour remplacer les colonnes contenant du texte, par des numéros.
# Pour chaque colonne, on attribue un numéro unique aux valeurs. S'il y a deux valeurs identiques dans la colonne
# contenant des chaînes de caractères, le numéro sera le même. Par exemple, 'LeHuffPost' aura un numéro unique dans
# la colonne channel_name_n.

X = yt
y = yt['nbrMotInsulte'] # variable de sortie
X.drop(['nbrMotInsulte'], axis = 1)

le_channel_name = LabelEncoder()
le_video_id_court = LabelEncoder()
le_video_id = LabelEncoder()
# le_channel_id = LabelEncoder() # on n'utilise pas channel_id_n car channel_name_n contient déjà toutes les informations.
le_categorie_new = LabelEncoder()
le_categ_inst = LabelEncoder()

# Dans ce qui suit, on rajoute les colonnes numériques, et supprimons les colonnes chaînes de caractère.
# nous n'avons pas besoin des colonnes video_id_n et video_id_court_n car toutes les valeurs sont différentes
# et n'aideront pas à la séparation au niveau d'un noeud dans l'arbre de décision.

X['channel_name_n'] = le_channel_name.fit_transform(X['channel_name'])
# X['video_id_court_n'] = le_video_id_court.fit_transform(X['video_id_court'])
# X['video_id_n'] = le_video_id.fit_transform(X['video_id'])
# X['channel_id_n'] = le_channel_id.fit_transform(X['channel_id'])
X['categorie_new_n'] = le_categorie_new.fit_transform(X['categorie_new'])
X['categ_inst_n'] = le_categ_inst.fit_transform(X['categ_inst'])

X = X.drop(['video_id_court', 'video_id', 'channel_id', 'channel_name',\
                  'categorie_new', 'categ_inst'], axis = 'columns')

In [80]:
# Les données d'entrainement et de test.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 1)

In [81]:
# On construit le modèle d'arbre de décision + fit sur le training set.

model = tree.DecisionTreeRegressor()
model.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [82]:
y_pred = model.predict(X_test) # y_prediction

In [85]:
# Validating the model ?
# print(y_pred, y_test)

print(f'Mean absolute error is{mean_absolute_error(y_test, y_pred) : .2f}')

Mean absolute error is 0.13


In [ ]:
# Other metrics for validation ?
# print(f'Accuracy score is{accuracy_score(y_test, y_pred) : .2f}')
# print(f'F1-score is{f1_score(y_test, y_pred, average="macro") : .2f}')
# print(f'Precision is{precision_score(y_test, y_pred, average="macro"): .2f}')
# print(f'Recall is{recall_score(y_test, y_pred, average="macro"): .2f}')

In [90]:
# on essaye différents paramètres afin d'optimiser l'arbre de décision

# criteria = ['mse', 'friedman_mse', 'mae']
# splitting_criteria = ['best', 'random']
tree_depth = [i for i in range(4, 25)] # le mse est trop grand si la profondeur de 
                                        # l'arbre est trop faible et il faut faire attention à ne pas overffiter
sample_split = [i for i in range(2,10)]
sample_leaf = [i for i in range(1,6)]

In [91]:
# on garde les mêmes train set et test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 1)
best_depth = 0
best_min_samples_split = 0
best_min_samples_leaf = 0
best_mse = 10**4
for depth in tree_depth :
    for split_sample in sample_split :
        for leaf_sample in sample_leaf :
            model = tree.DecisionTreeRegressor(criterion='mse', splitter='best', max_depth=depth, min_samples_split=split_sample, min_samples_leaf=leaf_sample)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            if mean_squared_error(y_test, y_pred) < best_mse :
                best_mse = mean_squared_error(y_test, y_pred)
                best_depth = depth
                best_min_samples_split = split_sample
                best_min_samples_leaf = leaf_sample
            else :
                best_mse = best_mse
            print('( Tree depth :', depth, ', min_samples_split :', split_sample,\
                  ', min_samples_leaf :', leaf_sample, ')', 'Mean squared error :',\
                  mean_squared_error(y_test, y_pred))
print('( Best depth :', best_depth, ', best_min_samples_split :', best_min_samples_split,\
      ', best_min_samples_leaf :', best_min_samples_leaf, ')', 'Best mean squared error :',\
      best_mse)

( Tree depth : 4 , min_samples_split : 2 , min_samples_leaf : 1 ) Mean squared error : 182.98737801455744
( Tree depth : 4 , min_samples_split : 2 , min_samples_leaf : 2 ) Mean squared error : 413.33072106840604
( Tree depth : 4 , min_samples_split : 2 , min_samples_leaf : 3 ) Mean squared error : 413.33072106840604
( Tree depth : 4 , min_samples_split : 2 , min_samples_leaf : 4 ) Mean squared error : 558.4983823155546
( Tree depth : 4 , min_samples_split : 2 , min_samples_leaf : 5 ) Mean squared error : 321.363615055973
( Tree depth : 4 , min_samples_split : 3 , min_samples_leaf : 1 ) Mean squared error : 140.86173634005814
( Tree depth : 4 , min_samples_split : 3 , min_samples_leaf : 2 ) Mean squared error : 413.33072106840604
( Tree depth : 4 , min_samples_split : 3 , min_samples_leaf : 3 ) Mean squared error : 413.33072106840604
( Tree depth : 4 , min_samples_split : 3 , min_samples_leaf : 4 ) Mean squared error : 558.4983823155546
( Tree depth : 4 , min_samples_split : 3 , min_sam

( Tree depth : 5 , min_samples_split : 9 , min_samples_leaf : 4 ) Mean squared error : 520.8710057341157
( Tree depth : 5 , min_samples_split : 9 , min_samples_leaf : 5 ) Mean squared error : 282.57907375019937
( Tree depth : 6 , min_samples_split : 2 , min_samples_leaf : 1 ) Mean squared error : 72.50364254337693
( Tree depth : 6 , min_samples_split : 2 , min_samples_leaf : 2 ) Mean squared error : 370.23617084132286
( Tree depth : 6 , min_samples_split : 2 , min_samples_leaf : 3 ) Mean squared error : 365.26447682064463
( Tree depth : 6 , min_samples_split : 2 , min_samples_leaf : 4 ) Mean squared error : 510.13802550494773
( Tree depth : 6 , min_samples_split : 2 , min_samples_leaf : 5 ) Mean squared error : 274.6411839162648
( Tree depth : 6 , min_samples_split : 3 , min_samples_leaf : 1 ) Mean squared error : 81.17905678995669
( Tree depth : 6 , min_samples_split : 3 , min_samples_leaf : 2 ) Mean squared error : 362.9670832843855
( Tree depth : 6 , min_samples_split : 3 , min_samp

( Tree depth : 7 , min_samples_split : 9 , min_samples_leaf : 2 ) Mean squared error : 352.7906880602734
( Tree depth : 7 , min_samples_split : 9 , min_samples_leaf : 3 ) Mean squared error : 352.7906880602734
( Tree depth : 7 , min_samples_split : 9 , min_samples_leaf : 4 ) Mean squared error : 508.4164560036016
( Tree depth : 7 , min_samples_split : 9 , min_samples_leaf : 5 ) Mean squared error : 272.8780828638749
( Tree depth : 8 , min_samples_split : 2 , min_samples_leaf : 1 ) Mean squared error : 126.9616660504318
( Tree depth : 8 , min_samples_split : 2 , min_samples_leaf : 2 ) Mean squared error : 367.5537050021159
( Tree depth : 8 , min_samples_split : 2 , min_samples_leaf : 3 ) Mean squared error : 352.325172032253
( Tree depth : 8 , min_samples_split : 2 , min_samples_leaf : 4 ) Mean squared error : 507.5663926298113
( Tree depth : 8 , min_samples_split : 2 , min_samples_leaf : 5 ) Mean squared error : 272.48196051261107
( Tree depth : 8 , min_samples_split : 3 , min_samples_

( Tree depth : 9 , min_samples_split : 8 , min_samples_leaf : 5 ) Mean squared error : 272.4111401628125
( Tree depth : 9 , min_samples_split : 9 , min_samples_leaf : 1 ) Mean squared error : 361.9051524005761
( Tree depth : 9 , min_samples_split : 9 , min_samples_leaf : 2 ) Mean squared error : 361.8429732378258
( Tree depth : 9 , min_samples_split : 9 , min_samples_leaf : 3 ) Mean squared error : 361.8494427025547
( Tree depth : 9 , min_samples_split : 9 , min_samples_leaf : 4 ) Mean squared error : 507.4473365244887
( Tree depth : 9 , min_samples_split : 9 , min_samples_leaf : 5 ) Mean squared error : 272.4111401628125
( Tree depth : 10 , min_samples_split : 2 , min_samples_leaf : 1 ) Mean squared error : 39.71938734727979
( Tree depth : 10 , min_samples_split : 2 , min_samples_leaf : 2 ) Mean squared error : 347.3876616947602
( Tree depth : 10 , min_samples_split : 2 , min_samples_leaf : 3 ) Mean squared error : 352.3511254316153
( Tree depth : 10 , min_samples_split : 2 , min_samp

( Tree depth : 11 , min_samples_split : 8 , min_samples_leaf : 4 ) Mean squared error : 507.4484735778727
( Tree depth : 11 , min_samples_split : 8 , min_samples_leaf : 5 ) Mean squared error : 272.4012226998639
( Tree depth : 11 , min_samples_split : 9 , min_samples_leaf : 1 ) Mean squared error : 361.8893168227211
( Tree depth : 11 , min_samples_split : 9 , min_samples_leaf : 2 ) Mean squared error : 352.1073646862162
( Tree depth : 11 , min_samples_split : 9 , min_samples_leaf : 3 ) Mean squared error : 361.8348250008074
( Tree depth : 11 , min_samples_split : 9 , min_samples_leaf : 4 ) Mean squared error : 507.9565707221782
( Tree depth : 11 , min_samples_split : 9 , min_samples_leaf : 5 ) Mean squared error : 272.40147527258705
( Tree depth : 12 , min_samples_split : 2 , min_samples_leaf : 1 ) Mean squared error : 54.01077290145326
( Tree depth : 12 , min_samples_split : 2 , min_samples_leaf : 2 ) Mean squared error : 360.0144903959705
( Tree depth : 12 , min_samples_split : 2 , m

...

( Tree depth : 15 , min_samples_split : 7 , min_samples_leaf : 4 ) Mean squared error : 507.97233827204025
( Tree depth : 15 , min_samples_split : 7 , min_samples_leaf : 5 ) Mean squared error : 272.4013364539911
( Tree depth : 15 , min_samples_split : 8 , min_samples_leaf : 1 ) Mean squared error : 352.0982988196846
( Tree depth : 15 , min_samples_split : 8 , min_samples_leaf : 2 ) Mean squared error : 361.7232641150356
( Tree depth : 15 , min_samples_split : 8 , min_samples_leaf : 3 ) Mean squared error : 361.7418618102291
( Tree depth : 15 , min_samples_split : 8 , min_samples_leaf : 4 ) Mean squared error : 507.950817346701
( Tree depth : 15 , min_samples_split : 8 , min_samples_leaf : 5 ) Mean squared error : 272.40136151845974
( Tree depth : 15 , min_samples_split : 9 , min_samples_leaf : 1 ) Mean squared error : 361.6726616037685
( Tree depth : 15 , min_samples_split : 9 , min_samples_leaf : 2 ) Mean squared error : 352.2628336809864
( Tree depth : 15 , min_samples_split : 9 , m

( Tree depth : 17 , min_samples_split : 7 , min_samples_leaf : 3 ) Mean squared error : 361.6346094521968
( Tree depth : 17 , min_samples_split : 7 , min_samples_leaf : 4 ) Mean squared error : 507.9510602792439
( Tree depth : 17 , min_samples_split : 7 , min_samples_leaf : 5 ) Mean squared error : 272.4016285514533
( Tree depth : 17 , min_samples_split : 8 , min_samples_leaf : 1 ) Mean squared error : 361.89113580423617
( Tree depth : 17 , min_samples_split : 8 , min_samples_leaf : 2 ) Mean squared error : 361.7226371819786
( Tree depth : 17 , min_samples_split : 8 , min_samples_leaf : 3 ) Mean squared error : 352.11447176853534
( Tree depth : 17 , min_samples_split : 8 , min_samples_leaf : 4 ) Mean squared error : 507.44616897227655
( Tree depth : 17 , min_samples_split : 8 , min_samples_leaf : 5 ) Mean squared error : 272.40110701770067
( Tree depth : 17 , min_samples_split : 9 , min_samples_leaf : 1 ) Mean squared error : 361.8901892590357
( Tree depth : 17 , min_samples_split : 9 

( Tree depth : 19 , min_samples_split : 7 , min_samples_leaf : 2 ) Mean squared error : 361.6109713887484
( Tree depth : 19 , min_samples_split : 7 , min_samples_leaf : 3 ) Mean squared error : 352.23691466030414
( Tree depth : 19 , min_samples_split : 7 , min_samples_leaf : 4 ) Mean squared error : 507.97672714486225
( Tree depth : 19 , min_samples_split : 7 , min_samples_leaf : 5 ) Mean squared error : 272.4015649262636
( Tree depth : 19 , min_samples_split : 8 , min_samples_leaf : 1 ) Mean squared error : 361.7226676288813
( Tree depth : 19 , min_samples_split : 8 , min_samples_leaf : 2 ) Mean squared error : 352.20117303943255
( Tree depth : 19 , min_samples_split : 8 , min_samples_leaf : 3 ) Mean squared error : 361.7342004376281
( Tree depth : 19 , min_samples_split : 8 , min_samples_leaf : 4 ) Mean squared error : 507.8559578924959
( Tree depth : 19 , min_samples_split : 8 , min_samples_leaf : 5 ) Mean squared error : 272.4013875469465
( Tree depth : 19 , min_samples_split : 9 ,

( Tree depth : 21 , min_samples_split : 6 , min_samples_leaf : 5 ) Mean squared error : 272.40162855145337
( Tree depth : 21 , min_samples_split : 7 , min_samples_leaf : 1 ) Mean squared error : 352.25493306099827
( Tree depth : 21 , min_samples_split : 7 , min_samples_leaf : 2 ) Mean squared error : 352.0328920741652
( Tree depth : 21 , min_samples_split : 7 , min_samples_leaf : 3 ) Mean squared error : 352.13681367941587
( Tree depth : 21 , min_samples_split : 7 , min_samples_leaf : 4 ) Mean squared error : 507.4664607746312
( Tree depth : 21 , min_samples_split : 7 , min_samples_leaf : 5 ) Mean squared error : 272.4009797673211
( Tree depth : 21 , min_samples_split : 8 , min_samples_leaf : 1 ) Mean squared error : 352.2631230068969
( Tree depth : 21 , min_samples_split : 8 , min_samples_leaf : 2 ) Mean squared error : 361.72578116610447
( Tree depth : 21 , min_samples_split : 8 , min_samples_leaf : 3 ) Mean squared error : 361.7337987634502
( Tree depth : 21 , min_samples_split : 8 

( Tree depth : 23 , min_samples_split : 6 , min_samples_leaf : 3 ) Mean squared error : 350.61087225556116
( Tree depth : 23 , min_samples_split : 6 , min_samples_leaf : 4 ) Mean squared error : 507.87999689317314
( Tree depth : 23 , min_samples_split : 6 , min_samples_leaf : 5 ) Mean squared error : 272.4016835004809
( Tree depth : 23 , min_samples_split : 7 , min_samples_leaf : 1 ) Mean squared error : 361.7722113569358
( Tree depth : 23 , min_samples_split : 7 , min_samples_leaf : 2 ) Mean squared error : 361.7781091388909
( Tree depth : 23 , min_samples_split : 7 , min_samples_leaf : 3 ) Mean squared error : 361.8733095943894
( Tree depth : 23 , min_samples_split : 7 , min_samples_leaf : 4 ) Mean squared error : 507.47392173927875
( Tree depth : 23 , min_samples_split : 7 , min_samples_leaf : 5 ) Mean squared error : 272.40168639253494
( Tree depth : 23 , min_samples_split : 8 , min_samples_leaf : 1 ) Mean squared error : 361.89393049249685
( Tree depth : 23 , min_samples_split : 8

In [109]:
# K-fold pou le cross-validation, avec les meilleurs paramètres qu'on a trouvé ci-dessus

scores = []
from sklearn.model_selection import KFold
kf = KFold(n_splits = 6)
for train_index, test_index in kf.split(X) :
    # print(train_index, test_index)
    # print(type(train_index))
    X_train, X_test, y_train, y_test = X.to_numpy()[train_index, :], X.to_numpy()[test_index, :], y.to_numpy()[train_index], y.to_numpy()[test_index]
    model = tree.DecisionTreeRegressor(max_depth=10, min_samples_split=2, min_samples_leaf=1)
    model.fit(X_train, y_train)
    scores.append(model.score(X_test, y_test))
print('Score du modèle :', np.mean(scores))

Score du modèle : 0.7462910823302676


Avec plusieurs run du modèle DT, on s'aperçoit que le score varie entre 0.70 et 0.94.

RANDOM FOREST REGRESSOR

In [110]:
# RandomForestRegressor with cross-validation

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
random_forest_score = []
kf = KFold(n_splits = 6)
for train_index, test_index in kf.split(X) :
    X_train, X_test, y_train, y_test = X.to_numpy()[train_index, :], X.to_numpy()[test_index, :], y.to_numpy()[train_index], y.to_numpy()[test_index]
    rd = RandomForestRegressor(n_estimators=40, max_depth=10, min_samples_split=2, min_samples_leaf=1)
    rd.fit(X_train, y_train)
    random_forest_score.append(rd.score(X_test, y_test))
print(np.mean(random_forest_score))

0.911494301412968


Avec un algorithme Random Forest composé de 40 DT, le résultat paraît plus stable à chaque itération. On obtient un score d'environ 0.90 à chaque run.